# Ch10. 전처리 실습 - San Francisco Crime
---
* 날짜: 2022-04-14
* 이름: 고민수

**소개**

  - [산프란시스코 범죄 데이터셋](https://www.kaggle.com/c/sf-crime) 전처리를 진행 합니다.
  - [파이썬 데이터전처리 실습-San francisco crime](https://wikidocs.net/86627)를 참고하여 전처리를 진행 합니다.
  - 향후 이 코드를 이어서 사용하니 코드를 잘 정리해 주세요.


## 데이터 로드, 확인
---

01. csv 파일을 데이터프레임으로 읽기

* 압축파일 열어서 읽기

In [95]:
import zipfile
import pandas as pd

with zipfile.ZipFile('sf_crime/train.csv.zip') as thezip:
    with thezip.open('train.csv',mode='r') as train:
        train = pd.read_csv(train)
        
with zipfile.ZipFile('sf_crime/test.csv.zip') as thezip:
    with thezip.open('test.csv',mode='r') as test:
        test = pd.read_csv(test)

In [28]:
train.head(2)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599


In [29]:
test.head(2)

# test 데이터의 특성이 더 적음
# test 데이터와 겹치는 데이터는 함께 전처리

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432


02. 학습 데이터 셋의 상위 5개 항목 출력

In [13]:
train.head()

# Dates - 범죄 사건 발생 날짜 및 시간 정보
# Category - 범죄 사건 카테고리(이 값이 예측할 목표변수)
# Descript - 범죄 사건에 대한 자세한 설명
# DayOfWeek - 요일
# PdDistrict - 경찰서 구역 이름
# Resolution - 범죄 사건이 해결 된 방법
# Address - 범죄 사건의 대략적인 주소
# X - 경도
# Y - 위도

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


03. 학습 데이터 셋의 각 필드 데이터 분포(평균, 최소값, 최대값 등)를 살펴보기

In [14]:
train.describe()

,X,Y
count,878049.000000,878049.000000
mean,-122.422616,37.771020
std,0.030354,0.456893
min,-122.513642,37.707879
25%,-122.432952,37.752427
50%,-122.416420,37.775421
75%,-122.406959,37.784369
max,-120.500000,90.000000


04. 각 컬럼의 데이터 타입 확인하기

In [15]:
train.info()

# 결측치는 없음
# 쓸데없는 object 변수가 너무 많음
# Dates를 년 월 일 시간 으로 나눔
# DayOfWeek 또한 요일 category로 변환

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878049 entries, 0 to 878048
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Dates       878049 non-null  object 
 1   Category    878049 non-null  object 
 2   Descript    878049 non-null  object 
 3   DayOfWeek   878049 non-null  object 
 4   PdDistrict  878049 non-null  object 
 5   Resolution  878049 non-null  object 
 6   Address     878049 non-null  object 
 7   X           878049 non-null  float64
 8   Y           878049 non-null  float64
dtypes: float64(2), object(7)
memory usage: 60.3+ MB


05. 범죄의 범주 'Category'는 어떤 값이 있는지 확인하기

In [16]:
train['Category'].unique()

array(['WARRANTS', 'OTHER OFFENSES', 'LARCENY/THEFT', 'VEHICLE THEFT',
       'VANDALISM', 'NON-CRIMINAL', 'ROBBERY', 'ASSAULT', 'WEAPON LAWS',
       'BURGLARY', 'SUSPICIOUS OCC', 'DRUNKENNESS',
       'FORGERY/COUNTERFEITING', 'DRUG/NARCOTIC', 'STOLEN PROPERTY',
       'SECONDARY CODES', 'TRESPASS', 'MISSING PERSON', 'FRAUD',
       'KIDNAPPING', 'RUNAWAY', 'DRIVING UNDER THE INFLUENCE',
       'SEX OFFENSES FORCIBLE', 'PROSTITUTION', 'DISORDERLY CONDUCT',
       'ARSON', 'FAMILY OFFENSES', 'LIQUOR LAWS', 'BRIBERY',
       'EMBEZZLEMENT', 'SUICIDE', 'LOITERING',
       'SEX OFFENSES NON FORCIBLE', 'EXTORTION', 'GAMBLING', 'BAD CHECKS',
       'TREA', 'RECOVERED VEHICLE', 'PORNOGRAPHY/OBSCENE MAT'],
      dtype=object)

# 데이터 전처리

06. Dates 필드의 데이터타입을 'datetime64' 타입으로 변경하기

* split을 할 필요없이 datetime64로 변환하면 "년 월 일 시간" 을 다 얻어 올 수 있다.
* Dates 는 test 데이터에도 있기때문에 함께 전처리

07. 연, 월, 일, 요일, 시, 분값을 새로운 컬럼(year, month, day, dayofweek, hour, minute)을 생성해서 할당하기

In [30]:
def Dates_split(df):
    df['Dates'] = df['Dates'].astype('datetime64')
    df['year'] = df['Dates'].dt.year
    df['month'] = df['Dates'].dt.month
    df['day'] = df['Dates'].dt.day
    df['dayofweek'] = df['Dates'].dt.dayofweek
    df['hour'] = df['Dates'].dt.hour
    df['minute'] = df['Dates'].dt.minute

In [32]:
Dates_split(train)
train.head(2)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,year,month,day,dayofweek,hour,minute
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,2,23,53
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,2,23,53


In [33]:
Dates_split(test)
test.head(2)

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y,year,month,day,dayofweek,hour,minute
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051,2015,5,10,6,23,59
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432,2015,5,10,6,23,51


08. 범죄가 발생한 경도, 위도 값을 나타내는 X, Y 컬럼의 값을 더한 값을 담은 'X+Y' 컬럼, X, Y의 차이 값을 담은 'X-Y' 컬럼을 생성하기

In [36]:
def X_Y_cal(df):
    df["X+Y"]= df["X"]+df["Y"]
    df["X-Y"]= df["X"]-df["Y"]

In [37]:
X_Y_cal(train)
train.head(2)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,year,month,day,dayofweek,hour,minute,X+Y,X-Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,2,23,53,-84.651293,-160.20049
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,2,23,53,-84.651293,-160.20049


In [38]:
X_Y_cal(test)
test.head(2)

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y,year,month,day,dayofweek,hour,minute,X+Y,X-Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051,2015,5,10,6,23,59,-84.664537,-160.134639
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432,2015,5,10,6,23,51,-84.659091,-160.123955


09. n_days라는 컬럼을 생성하고, 일별 d-day를 계산하여 입력하기

* Dates 칼럼에서 date.min 값을 찾고 해당 데이터의 날짜와 비교

In [44]:
def n_days_cal(df):
    df["n_days"] = (df["Dates"].dt.date-df["Dates"].dt.date.min()).apply(lambda x: x.days)
    
    # df["Dates"].dt,date 에서 해당 데이터의 date
    # df["Dates"].dt.date.min() 에서 dates 칼럼에서 가장 낮은 값
    # 그 차이를 출력하면 ( )days 값을 얻을수있음
    # apply x: x.days 를 통해 ( )days 의 수치값을 얻음

In [45]:
n_days_cal(train)
train.head(2)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,year,month,day,dayofweek,hour,minute,X+Y,X-Y,n_days
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,2,23,53,-84.651293,-160.20049,4510
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,2,23,53,-84.651293,-160.20049,4510


In [46]:
n_days_cal(test)
test.head(2)

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y,year,month,day,dayofweek,hour,minute,X+Y,X-Y,n_days
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051,2015,5,10,6,23,59,-84.664537,-160.134639,4512
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432,2015,5,10,6,23,51,-84.659091,-160.123955,4512


10. 변수 y를 선언해서 학습할 목표변수(=종속변수)인 Category 필드값을 담기

In [47]:
y = train['Category']

11. 연도별(year) 범죄 발생 횟수를 알아보기

In [52]:
train.groupby(["year"])["Descript"].count()

year
2003    73902
2004    73422
2005    70779
2006    69909
2007    68015
2008    70174
2009    69000
2010    66542
2011    66619
2012    71731
2013    75606
2014    74766
2015    27584
Name: Descript, dtype: int64

12. 월별(month) 범죄 발생 횟수를 알아보기

In [54]:
train.groupby(["month"])["Descript"].count()

month
1     73536
2     70813
3     76320
4     78096
5     79644
6     70892
7     69971
8     68540
9     71982
10    80274
11    72975
12    65006
Name: Descript, dtype: int64

13. 요일별(DayOfWeek) 범죄 발생 횟수를 알아보기

In [56]:
train.groupby(["DayOfWeek"])["Descript"].count()

DayOfWeek
Friday       133734
Monday       121584
Saturday     126810
Sunday       116707
Thursday     125038
Tuesday      124965
Wednesday    129211
Name: Descript, dtype: int64

14. 카테고리별(Category) 범죄 발생 횟수를 알아보기

In [57]:
train.groupby(["Category"])["Descript"].count()

Category
ARSON                            1513
ASSAULT                         76876
BAD CHECKS                        406
BRIBERY                           289
BURGLARY                        36755
DISORDERLY CONDUCT               4320
DRIVING UNDER THE INFLUENCE      2268
DRUG/NARCOTIC                   53971
DRUNKENNESS                      4280
EMBEZZLEMENT                     1166
EXTORTION                         256
FAMILY OFFENSES                   491
FORGERY/COUNTERFEITING          10609
FRAUD                           16679
GAMBLING                          146
KIDNAPPING                       2341
LARCENY/THEFT                  174900
LIQUOR LAWS                      1903
LOITERING                        1225
MISSING PERSON                  25989
NON-CRIMINAL                    92304
OTHER OFFENSES                 126182
PORNOGRAPHY/OBSCENE MAT            22
PROSTITUTION                     7484
RECOVERED VEHICLE                3138
ROBBERY                         23000
RUN

15. null값이 포함된 데이터 필드가 있는지 확인하기

In [64]:
train.isnull().sum()

Dates         0
Category      0
Descript      0
DayOfWeek     0
PdDistrict    0
Resolution    0
Address       0
X             0
Y             0
year          0
month         0
day           0
dayofweek     0
hour          0
minute        0
X+Y           0
X-Y           0
n_days        0
dtype: int64

16. 'Dates','Category','Descript','DayOfWeek','Resolution' 컬럼 삭제하기

In [77]:
def drop_columns(df):
    if "Dates" in df.columns:
        df.drop(['Dates'], axis=1, inplace=True)
    if "Category" in df.columns:
        df.drop(['Category'], axis=1, inplace=True)
    if "Descript" in df.columns:
        df.drop(['Descript'], axis=1, inplace=True)
    if "DayOfWeek" in df.columns:
        df.drop(['DayOfWeek'], axis=1, inplace=True)
    if "Resolution" in df.columns:
        df.drop(['Resolution'], axis=1, inplace=True)
    if "Id" in df.columns:
        df.drop(['Id'], axis=1, inplace=True)

In [75]:
drop_columns(train)
train.head(3)

,PdDistrict,Address,X,Y,year,month,day,dayofweek,hour,minute,X+Y,X-Y,n_days
0,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,2,23,53,-84.651293,-160.200490,4510
1,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,2,23,53,-84.651293,-160.200490,4510
2,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,5,13,2,23,33,-84.623949,-160.224777,4510


In [79]:
drop_columns(test)
test.head(3)

,PdDistrict,Address,X,Y,year,month,day,dayofweek,hour,minute,X+Y,X-Y,n_days
0,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051,2015,5,10,6,23,59,-84.664537,-160.134639,4512
1,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432,2015,5,10,6,23,51,-84.659091,-160.123955,4512
2,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212,2015,5,10,6,23,50,-84.633790,-160.218214,4512


# 데이터 전처리 파이프라인

In [80]:
def Preprocessing_sfcrime(df):
    Dates_split(df)
    X_Y_cal(df)
    n_days_cal(df)
    drop_columns(df)